In [9]:
import pyaudio
import wave
import speech_recognition as sr
from nltk.corpus import cmudict
import nltk

# Step 1: Load Pre-saved Pronunciation
def load_pronunciation(file_path):
    try:
        with open(file_path, 'r') as file:
            pronunciation = file.read().lower().strip()
        return pronunciation
    except FileNotFoundError:
        print(f"File not found at {file_path}. Please save the pronunciation first.")
        return None

# Example usage
saved_pronunciation = load_pronunciation("Abhinav_Sharma_pronunciation.txt")
if saved_pronunciation is None:
    # Handle the case where the pronunciation file is not found
    # You may want to prompt the user to save the pronunciation first
    pass
else:
    print(f"Loaded Pronunciation: {saved_pronunciation}")

# Step 2: Record User's Speech
def record_user_speech(file_path, duration=5):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Recording...")

    frames = []

    for i in range(0, int(RATE / CHUNK * duration)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording complete.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(file_path, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

# Example usage
user_audio_file = "user_speech.wav"
record_user_speech(user_audio_file)

# Step 3: Recognize User's Speech
def recognize_user_speech(file_path):
    recognizer = sr.Recognizer()

    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio_data)
        return text.lower()
    except sr.UnknownValueError:
        return "Unable to recognize speech"
    except sr.RequestError as e:
        return f"Error with the speech recognition service; {e}"

# Example usage
user_recognized_text = recognize_user_speech(user_audio_file)
print(f"User's Recognized Text: {user_recognized_text}")


# Step 4: Compare Pronunciations and Calculate Score
def calculate_pronunciation_score(actual, predicted):
    actual_words = actual.lower().split()
    predicted_words = predicted.lower().split()

    # Calculate scores for both original order and reversed order
    score_original_order = calculate_score_for_order(actual_words, predicted_words)
    score_reversed_order = calculate_score_for_order(actual_words[::-1], predicted_words)

    # Deduct points for the reversed order
    penalty_factor = 0.8
    final_score = score_original_order - penalty_factor * score_reversed_order

    # Ensure the final score is within [0, 1]
    final_score = max(0.0, min(final_score, 1.0))

    return final_score

def calculate_score_for_order(actual_order, predicted_order):
    common_words = set(actual_order) & set(predicted_order)
    # Adjust scoring sensitivity based on the number of common words
    score = len(common_words) / max(len(actual_order), len(predicted_order)) if len(actual_order) > 0 else 0.0
    return score

# Example usage
actual_pronunciation = "Abhinav Sharma"
user_recognized_text = "Sharma Abhinav"  # Example user input

score = calculate_pronunciation_score(actual_pronunciation, user_recognized_text)
print(f"Pronunciation Score: {score}")





Loaded Pronunciation: purakh
Recording...
Recording complete.
User's Recognized Text: purakh
Pronunciation Score: 0.19999999999999996
